In [1]:
# import package
import os
from datetime import datetime

In [5]:
os.path.dirname(os.path.('__file__'))

''

In [6]:

os.path.relpath('/path/to/file.ext', os.getcwd())

'../../../../../../path/to/file.ext'

In [7]:
os.getcwd()

'/home/ay/DataEng/projects/DS_RedditEndToEnd/notebooks'

In [8]:
from pathlib import Path

In [9]:
 Path.cwd() / 'config.ini'

PosixPath('/home/ay/DataEng/projects/DS_RedditEndToEnd/notebooks/config.ini')

In [10]:
Path.home() / 'config.ini'

PosixPath('/home/ay/config.ini')